<a href="https://colab.research.google.com/github/Computerglassformedandsurg/BRATS_SEGMENATION/blob/main/Brats_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before uploading the code in the github repository, create a new file in the repository


# Get the zip file containing data in google drive and unzip it to google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Colab Notebooks/brats20-dataset-training-validation.zip'

!unzip brats20-dataset-training-validation.zip -d /content/data

# Installing and Importing libraries

In [ ]:
!pip install nibabel

In [ ]:
import numpy as np
import nibabel as nib
import os
import glob
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imwrite

from sklearn.preprocessing import MinMaxScaler

In [ ]:
Train_data_path = "/content/data/BraTS2020_TrainingData/"
# loading nif file as npy

# file_path
file_path = os.path.join(Train_data_path,'MICCAI_BraTS2020_TrainingData', 'BraTS20_Training_355','BraTS20_Training_355_flair.nii' )
test_image_flair = nib.load(file_path).get_fdata()
print(test_image_flair.max())

1854.603271484375


Apply Minmax scaler

reshapes the 3D data into a 2D matrix (combines all dimensions, keeping the last dimension as it is), applies the scaler, and then reshapes the data back to its original 3D shape.

In [ ]:
scaler = MinMaxScaler()
test_image_flair1 = scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)

***References***

*   **Code:**https://github.com/bnsreenu/python_for_microscopists/tree/master/231_234_BraTa2020_Unet_segmentation
* **Video:**https://www.youtube.com/watch?v=oB35sV1npVI&list=PLZsOBAyNTZwYgF8O1bTdV-lBdN55wLHDr&index=2

# Please open and read this before going further

The below cells are to download datasets from kaggle and upload it into google colab and then to google drive as zip file.

Everytime we revisit, we just have to run the first part of cells which get the zip file containing data from the drive and unzip the folder into the google colab

# Initiating new Kaggle API to download the Brats Data

In [ ]:
# !pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jaisilrose","key":"04e333b704a4341400adda79428f11fe"}'}

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# Kaggle to Colab- Method 1

---



In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

# print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brats20-dataset-training-validation' dataset.
Path to dataset files: /kaggle/input/brats20-dataset-training-validation


# Kaggle to Colab - Method 2


In [ ]:
# !kaggle datasets download -d awsaf49/brats20-dataset-training-validation


Dataset URL: https://www.kaggle.com/datasets/awsaf49/brats20-dataset-training-validation
License(s): CC0-1.0
 98% 4.09G/4.16G [02:03<00:02, 27.9MB/s]
100% 4.16G/4.16G [02:03<00:00, 36.1MB/s]


In [ ]:
# !unzip brats20-dataset-training-validation.zip -d /content/data

Archive:  brats20-dataset-training-validation.zip
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_flair.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1ce.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t2.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002/BraTS20_Training_002_flair.nii  
  inflating: /content/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002/BraTS20_Training_002_seg.nii  
  inflating: /content/d

# Saving the zip file containing data in the google drive




In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# import shutil

# source_path = '/content/brats20-dataset-training-validation.zip'
# destination_path = '/content/drive/My Drive/Colab Notebooks/brats20-dataset-training-validation.zip'

# # Move the file
# shutil.move(source_path, destination_path)

# print(f"'{source_path}' moved to Drive at '{destination_path}'")

'/content/brats20-dataset-training-validation.zip' moved to Drive at '/content/drive/My Drive/Colab Notebooks/brats20-dataset-training-validation.zip'
